In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import numpy as np
from tcvae import predict, dataset, localconfig


In [ ]:
conf = localconfig.LocalConfig()
conf.load_config_from_file(
    "checkpoints/cnn_ae5_2d/Default_cnn_ae5_2d.json"
)
conf.dataset_dir = "complete_dataset"
conf.batch_size = 1

_, _, test = dataset.get_dataset(conf)
test_iter = iter(test)

In [ ]:
_model = predict.load_model(
    conf,
    "checkpoints/cnn_ae5_2d/79_cnn_ae5_2d_0.01495.h5"
)

## Export Some Examples

In [ ]:
import soundfile as sf
import tsms


def write_audio(h_freq, h_mag, h_phase, conf, audio_path):
    audio = tsms.core.harmonic_synthesis(
        h_freq, h_mag, h_phase, conf.sample_rate, conf.frame_size)
    audio = np.squeeze(audio.numpy())
    audio = audio / np.max(np.abs(audio))
    sf.write(audio_path, audio, samplerate=conf.sample_rate)


for i in range(0, 20):
    batch = next(test_iter)
    (h_freq_true, h_mag_true, h_phase_true,
     h_freq_pred, h_mag_pred, h_phase_pred) = predict.get_prediction(_model, batch, conf)

    true_path = os.path.join(os.getcwd(), "predictions", f"{i}_true.wav")
    pred_path = os.path.join(os.getcwd(), "predictions", f"{i}_pred.wav")

    write_audio(h_freq_true, h_mag_true, h_phase_true, conf, true_path)
    write_audio(h_freq_pred, h_mag_pred, h_phase_pred, conf, pred_path)

    print(i)

In [1]:
import numpy as np
from sound_generator import SoundGenerator
from IPython.display import Audio
from tcvae import dataset

config = "checkpoints/cnn_ae5_2d/Default_cnn_ae5_2d.json"
checkpoint = "checkpoints/cnn_ae5_2d/79_cnn_ae5_2d_0.01495.h5"

sg = SoundGenerator(config)
sg.load_model(checkpoint)

sg.conf.dataset_dir = "complete_dataset"
train, _, _ = dataset.get_dataset(sg.conf)

INFO:root:Initializing SoundGenerator
INFO:root:SoundGenerator initialized
INFO:root:Creating complete model from config


Creating Auto Encoder


INFO:root:Loading pretrained weights for complete model
INFO:root:Complete model loaded
INFO:root:Creating decoder
INFO:root:Decoder created


In [2]:
train_iter = iter(train)

In [3]:
batch = next(train_iter)
z = sg.encoder.predict(batch["h"])

print(z)

print(np.mean(z))
print(np.min(z))
print(np.max(z))
print(np.std(z))


[[1.0609375 1.2616953 0.5713314 1.0012288 1.4979545]]
1.0786295
0.5713314
1.4979545
0.3075717


In [4]:
measures = batch["measures"]
print(measures)

tf.Tensor(
[[0.9971938  0.65744567 0.12538967 0.01675248 0.02236341 0.003996
  0.22877122 0.02519462 0.6991602  0.29892266 0.29033065]], shape=(1, 11), dtype=float32)


In [13]:
pred = sg.get_prediction({
    "z": None, # z[0].tolist(),
    "measures": measures.numpy()[0].tolist(),
    "pitch": 42,
    "velocity": 100
})

87


In [14]:
audio = pred["audio"]
Audio(audio, rate=16000)

In [24]:
pred["z"]

[-2.1677251774040167,
 -2.2009313405638093,
 2.76934910754696,
 0.2627049180160301,
 4.6762568405942]

In [25]:
pred["measures"]

[0.2825229369195575,
 0.862190980888438,
 0.05869520677498814,
 0.7105248969803164,
 -0.47228500590957967,
 -0.2826879948034669,
 0.2154043564434022,
 -0.7093931763034094,
 -0.1222837169921461,
 1.5441439261725305,
 -0.0692045076970951]